In [2]:
import numpy as np

In [3]:
def initialize_parameters(layer_dim, seed = 3):

    """
    Arguments:
    layer_dim: Array with the dimensions of each layer of the network
    seed: random seed
    
    Return:
    W: weight matrix (n_l, n_l-1)
    b: bias vector (n_l, 1)
    """

    np.random.seed(seed)
    
    parameters = {}
    
    L = len(layer_dim) # Layers in the network
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dim[l], layer_dim[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dim[l], 1))

    return parameters

**This is an example for Neural Network**


<img src="Images/1.png" width="600">   

In [26]:
# Test with the sample NN
parameters = initialize_parameters([3,5,5,3,1])
print(parameters)
for i, j in enumerate(parameters):
    print(f"Dimensions of {j} is: {parameters[j].shape}")

{'W1': array([[ 0.01788628,  0.0043651 ,  0.00096497],
       [-0.01863493, -0.00277388, -0.00354759],
       [-0.00082741, -0.00627001, -0.00043818],
       [-0.00477218, -0.01313865,  0.00884622],
       [ 0.00881318,  0.01709573,  0.00050034]]), 'b1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]]), 'W2': array([[-0.00404677, -0.0054536 , -0.01546477,  0.00982367, -0.01101068],
       [-0.01185047, -0.0020565 ,  0.01486148,  0.00236716, -0.01023785],
       [-0.00712993,  0.00625245, -0.00160513, -0.00768836, -0.00230031],
       [ 0.00745056,  0.01976111, -0.01244123, -0.00626417, -0.00803766],
       [-0.02419083, -0.00923792, -0.01023876,  0.01123978, -0.00131914]]), 'b2': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]]), 'W3': array([[-0.01623285,  0.00646675, -0.00356271, -0.01743141, -0.0059665 ],
       [-0.00588594, -0.00873882,  0.00029714, -0.02248258, -0.00267762],
       [ 0.01013183,  0.00852798,  0.01108187,  0.01119391,  0.01487543]]